In [ ]:
import time
import gspread
import requests

from oauth2client.service_account import ServiceAccountCredentials

# Define exchange URLs
exchange_urls = {
    "Wazirx": "https://api.wazirx.com/api/v2/tickers",
    "CoinDCX": "https://api.coindcx.com/exchange/ticker",
    "Coinswitch": "https://cs-india.coinswitch.co/api/v2/external/csk_website/currencies",
    "Mudrex": "https://mudrex.com/api/v1/web/coins?sort=price&filter=price&order=desc&min=0&max=9007199254740991&size=500&offset=0",
}

# Define column names
column_names_wazirx = ["Base Unit", "Quote Unit", "Sell", "Buy", "Name"]

# Google Sheets credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/pawan/Downloads/creds.json', scope)
gc = gspread.authorize(credentials)

# Google Sheets document URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1k9vZJsj8vSzxu7r7rQyrcBj-2Cnw5lTNPVgbY59Sr3I/edit?pli=1#gid=0'


def dollar():
    try:
        api_endpoint = "https://api.exchangerate-api.com/v4/latest/USD"
        # Fetch data from the API
        response = requests.get(api_endpoint)
        response.raise_for_status()
        data = response.json()
        dollar_rate = data["rates"]["INR"]  # Change "INR" to the currency code you desire
        return dollar_rate

    except requests.exceptions.RequestException as e:
            print(f"Failed to fetch data: {e}")

def fetch_exchange_data(url, retries=3, retry_delay=3):
    for _ in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch data: {e}")
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
    print(f"Maximum retries reached. Unable to fetch data from {url}.")
    return None

def update_sheet(data, header=None, sheet_name='Sheet1', retries=100, retry_delay=3):
    worksheet = gc.open_by_url(sheet_url).worksheet(sheet_name)
    for _ in range(retries):
        try:
            # Clear old data before updating new data
            worksheet.clear()

            if header:
                worksheet.append_row(header)

            for item in data:
                worksheet.append_row(item)

            # If the write operation is successful, break the loop
            break
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429:  # Check if the error is due to quota exceeded
                print("Quota exceeded. Retrying after delay...")
                time.sleep(retry_delay)
            else:
                print(f"API Error: {e}")
                break
    else:
        print("Maximum retries reached. Unable to update sheet.")


def update_wazirx_sheet():
    data = fetch_exchange_data(exchange_urls.get("Wazirx"))
    if data:
        sorted_data = sorted(data.items(), key=lambda x: float(x[1].get('buy', 0)), reverse=True)[:20]
        parsed_data = [[value.get('base_unit', ''),value.get('name', ''), value.get('buy', '')] for _, value in sorted_data]
        update_sheet(parsed_data, ["Token", "Name", "Price in INR"], sheet_name="Wazirx")

def update_coindcx_sheet():
    data = fetch_exchange_data(exchange_urls.get("CoinDCX"))
    if data:
        parsed_data = []
        sorted_data = sorted(data, key=lambda x: float(x.get('ask', 0)), reverse=True)[:20]
        for market_data in sorted_data:
            if market_data.get('market', '') != "BTCINR_insta":
                # Format numerical values without decimal values
                last_price = "{:.1f}".format(float(market_data.get('last_price', '')))
                parsed_data.append([market_data.get('market', ''),
                                     last_price])
        update_sheet(parsed_data, ["Token", "Price in INR"], sheet_name="CoinDCX")


def update_coinswitch_sheet():
    data = fetch_exchange_data(exchange_urls.get("Coinswitch"))
    if data and 'data' in data:
        currencies = data['data']['currencies']
        sorted_currencies = sorted(currencies, key=lambda x: float(x.get('price', '').replace(',', '')), reverse=True)[:20]
        parsed_data = [[currency.get('currency', ''), currency.get('currency_name', ''),
                        currency.get('price', '').replace(',', '')] for currency in sorted_currencies]
        update_sheet(parsed_data, ["Token", "Name", "Price in INR"], sheet_name="Coinswitch")

def update_mudrex_sheet():
    data = fetch_exchange_data(exchange_urls.get("Mudrex"))
    if data and 'data' in data:
        coins_data = data['data']
        sorted_coins = sorted(coins_data, key=lambda x: float(x.get('price_info', {}).get('price', 0)), reverse=True)[:20]
        parsed_data = []
        for coin in sorted_coins:
            token = coin.get('token', '')
            name = coin.get('name', '')
            price_in_dollars = float(coin.get('price_info', {}).get('price', ''))
            price_in_inr = price_in_dollars * dollar()
            parsed_data.append([token, name, f"{price_in_inr:.1f} INR"])  # Format the price to one decimal place
        update_sheet(parsed_data, ["Token", "Name", "Price in INR"], sheet_name="Mudrex")

def main():
    try:
            
        while True:
            update_wazirx_sheet()
            time.sleep(10) 

            update_coindcx_sheet()
            time.sleep(10)  

            update_coinswitch_sheet()
            time.sleep(10) 

            update_mudrex_sheet()
            time.sleep(30)  

    except KeyboardInterrupt:
        print("Process interrupted. Exiting...")

if __name__ == "__main__":
    main()

